In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

Using TensorFlow backend.


In [2]:
# The data, split between train and test sets:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
from fedml import Alliance, AllianceMember,KerasSequentialBaseLearner, FedAveragingClassifier
from read_data import read_training_data
import numpy as np

alliance = Alliance()
M=10
def split_data(x,y,fractions):
    s = len(x)
    new_order = np.random.permutation(s)
    x = x[new_order]
    y = y[new_order]
    if type(fractions) is int:
        fraction_splits = np.int32(np.linspace(0,s,fractions+1))
        nr = fractions
    elif type(fractions)==np.ndarray or type(fractions)==list:
        parts = np.array(fractions)*s/np.sum(fractions)
        fraction_splits = np.int32([np.sum(parts[:i]) for i in range(len(parts)+1)])
        nr = len(parts)
        
    x_list = []
    y_list = []
    for i in range(nr):
        x_list.append(x[fraction_splits[i]:fraction_splits[i+1]]) 
        y_list.append(y[fraction_splits[i]:fraction_splits[i+1]])

    return x_list,y_list
F = [10,10,8,8,6,6,4,4,2,2,1,1]
M = len(F)
x_train_splits, y_train_splits = split_data(x_train,y_train,fractions=F)
x_test_splits, y_test_splits = split_data(x_test,y_test,fractions=F)
print("x_train shape: ", np.array(x_train).shape)
tot_len = 0
for split in x_train_splits:
    print("x_train_splits shape: ", split.shape)
    tot_len += len(split)
print("tot len: ", tot_len)
alliance.set_validation_dataset(x_test,y_test)

classes = np.arange(10)
i = 1
member = AllianceMember(x_train_splits[i], y_train_splits[i], classes=classes)
alliance.add_member(member)
epochs = int(1563*500/int(np.ceil(len(x_train_splits[i])/32)))
print("epochs: ", epochs)

x_train shape:  (50000, 32, 32, 3)
x_train_splits shape:  (8064, 32, 32, 3)
x_train_splits shape:  (8065, 32, 32, 3)
x_train_splits shape:  (6451, 32, 32, 3)
x_train_splits shape:  (6452, 32, 32, 3)
x_train_splits shape:  (4838, 32, 32, 3)
x_train_splits shape:  (4839, 32, 32, 3)
x_train_splits shape:  (3226, 32, 32, 3)
x_train_splits shape:  (3226, 32, 32, 3)
x_train_splits shape:  (1613, 32, 32, 3)
x_train_splits shape:  (1613, 32, 32, 3)
x_train_splits shape:  (806, 32, 32, 3)
x_train_splits shape:  (807, 32, 32, 3)
tot len:  50000


In [4]:
from cifarmodel import KerasSequentialCifar

base_learner = KerasSequentialCifar()
fedavg_model = FedAveragingClassifier(alliance=alliance,base_learner=base_learner)


In [10]:
parameters = {"nr_global_iterations": 1801, "nr_local_iterations":1, "training_steps": None, "data_augmentation": True, "model_size_averaging": True} 
training_loss, test_loss = fedavg_model.fit(parameters)

fit starts!
else: alliance temp model:  <cifarmodel.KerasSequentialCifar object at 0x7f0a3cf7b2b0>
global epoch:  0
virtual memory used:  63.8 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 14s 56ms/step - loss: 0.2122 - accuracy: 0.9263
training time:  16.1599 s.
average model starts
average model ends:  0.1865 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.792 0.8   0.786]
global epoch:  1
virtual memory used:  64.7 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 49ms/step - loss: 0.2116 - accuracy: 0.9306
training time:  12.4008 s.
average model starts
average model ends:  0.1071 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.8   0.786 0.794]
global epoch:  2
virtual memory used:  65.5 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 49ms/step - loss: 0.2149 - accur

test_loss:  [0.202 0.364 0.409 ... 0.791 0.798 0.775]
global epoch:  23
virtual memory used:  69.5 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 46ms/step - loss: 0.2128 - accuracy: 0.9318
training time:  11.7944 s.
average model starts
average model ends:  0.0393 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.798 0.775 0.793]
global epoch:  24
virtual memory used:  70.1 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 48ms/step - loss: 0.2328 - accuracy: 0.9244
training time:  12.2444 s.
average model starts
average model ends:  0.1186 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.775 0.793 0.79 ]
global epoch:  25
virtual memory used:  68.6 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 13s 50ms/step - loss: 0.2127 - accuracy: 0.9307
training time:  12.7315 s.
ave

252/252 [==============================] - 13s 50ms/step - loss: 0.2143 - accuracy: 0.9317
training time:  12.7541 s.
average model starts
average model ends:  0.1559 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.799 0.794 0.786]
global epoch:  47
virtual memory used:  66.0 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 14s 54ms/step - loss: 0.2281 - accuracy: 0.9302
training time:  13.8329 s.
average model starts
average model ends:  0.0497 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.794 0.786 0.79 ]
global epoch:  48
virtual memory used:  66.6 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 13s 51ms/step - loss: 0.2180 - accuracy: 0.9270
training time:  12.8979 s.
average model starts
average model ends:  0.0453 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.786 0.79  0.805]
global epoch:  49
virtual memory used:  67.2 

252/252 [==============================] - 12s 49ms/step - loss: 0.2204 - accuracy: 0.9281
training time:  12.5448 s.
average model starts
average model ends:  0.0579 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.787 0.799 0.787]
global epoch:  70
virtual memory used:  68.8 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 48ms/step - loss: 0.2175 - accuracy: 0.9296
training time:  12.165 s.
average model starts
average model ends:  0.0567 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.799 0.787 0.788]
global epoch:  71
virtual memory used:  69.6 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 11s 44ms/step - loss: 0.2326 - accuracy: 0.9209
training time:  11.1099 s.
average model starts
average model ends:  0.0954 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.787 0.788 0.796]
global epoch:  72
virtual memory used:  70.1 %

252/252 [==============================] - 14s 54ms/step - loss: 0.2221 - accuracy: 0.9278
training time:  13.6424 s.
average model starts
average model ends:  0.0585 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.795 0.788 0.794]
global epoch:  93
virtual memory used:  73.6 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 13s 51ms/step - loss: 0.2149 - accuracy: 0.9311
training time:  12.9872 s.
average model starts
average model ends:  0.0538 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.788 0.794 0.803]
global epoch:  94
virtual memory used:  65.0 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 47ms/step - loss: 0.2104 - accuracy: 0.9325
training time:  11.9938 s.
average model starts
average model ends:  0.053 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.794 0.803 0.795]
global epoch:  95
virtual memory used:  66.0 %

...

252/252 [==============================] - 12s 49ms/step - loss: 0.1580 - accuracy: 0.9477
training time:  12.4564 s.
average model starts
average model ends:  0.0376 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.803 0.799 0.797]
global epoch:  1703
virtual memory used:  90.3 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 49ms/step - loss: 0.1504 - accuracy: 0.9506
training time:  12.4328 s.
average model starts
average model ends:  0.0668 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.799 0.797 0.802]
global epoch:  1704
virtual memory used:  90.9 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 11s 43ms/step - loss: 0.1594 - accuracy: 0.9447
training time:  10.8825 s.
average model starts
average model ends:  0.1071 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.797 0.802 0.799]
global epoch:  1705
virtual memory used: 

252/252 [==============================] - 11s 45ms/step - loss: 0.1630 - accuracy: 0.9466
training time:  11.5455 s.
average model starts
average model ends:  0.0637 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.794 0.794 0.801]
global epoch:  1726
virtual memory used:  86.7 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 48ms/step - loss: 0.1581 - accuracy: 0.9483
training time:  12.1327 s.
average model starts
average model ends:  0.0429 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.794 0.801 0.796]
global epoch:  1727
virtual memory used:  87.4 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 13s 50ms/step - loss: 0.1744 - accuracy: 0.9452
training time:  12.8202 s.
average model starts
average model ends:  0.0623 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.801 0.796 0.803]
global epoch:  1728
virtual memory used: 

252/252 [==============================] - 11s 43ms/step - loss: 0.1539 - accuracy: 0.9511
training time:  10.9201 s.
average model starts
average model ends:  0.075 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.804 0.791 0.798]
global epoch:  1749
virtual memory used:  90.3 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 11s 45ms/step - loss: 0.1547 - accuracy: 0.9511
training time:  11.399 s.
average model starts
average model ends:  0.0592 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.791 0.798 0.806]
global epoch:  1750
virtual memory used:  90.9 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 11s 44ms/step - loss: 0.1665 - accuracy: 0.9435
training time:  11.3232 s.
average model starts
average model ends:  0.0395 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.798 0.806 0.802]
global epoch:  1751
virtual memory used:  8

252/252 [==============================] - 11s 44ms/step - loss: 0.1674 - accuracy: 0.9432
training time:  11.2343 s.
average model starts
average model ends:  0.0858 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.796 0.801 0.799]
global epoch:  1772
virtual memory used:  87.0 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 11s 45ms/step - loss: 0.1472 - accuracy: 0.9525
training time:  11.6062 s.
average model starts
average model ends:  0.0795 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.801 0.799 0.801]
global epoch:  1773
virtual memory used:  88.0 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 47ms/step - loss: 0.1576 - accuracy: 0.9483
training time:  12.0918 s.
average model starts
average model ends:  0.0877 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.799 0.801 0.803]
global epoch:  1774
virtual memory used: 

252/252 [==============================] - 13s 50ms/step - loss: 0.1649 - accuracy: 0.9452
training time:  12.8445 s.
average model starts
average model ends:  0.1435 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.796 0.799 0.802]
global epoch:  1795
virtual memory used:  90.8 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 13s 53ms/step - loss: 0.1465 - accuracy: 0.9528
training time:  13.4215 s.
average model starts
average model ends:  0.0424 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.799 0.802 0.805]
global epoch:  1796
virtual memory used:  91.5 %
training starts model size:  8064
training steps:  None
Epoch 1/1
252/252 [==============================] - 12s 48ms/step - loss: 0.1591 - accuracy: 0.9483
training time:  12.3364 s.
average model starts
average model ends:  0.0812 s.
alliance_test_loss
test_loss:  [0.202 0.364 0.409 ... 0.802 0.805 0.803]
global epoch:  1797
virtual memory used: 

In [9]:
itterations = len(fedavg_model.test_loss)
tot_epochs = int(1563*500/int(np.ceil(len(x_train_splits[0])/32)))
epochs_left = tot_epochs - itterations
print("itterations: ", itterations, ", tot epochs: ", tot_epochs, ", epochs_left: ", epochs_left)

itterations:  1300 , tot epochs:  3101 , epochs_left:  1801


In [11]:
from matplotlib import pylab as plt
import pickle
pickle.dump(fedavg_model.test_loss,open('test_loss_3101e_bigestmember_12members_diffsizes.p','wb'))


plt.plot(fedavg_model.test_loss)
plt.savefig('accuracy_progress_b')
plt.show()
